In [32]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import cv2
import numpy as np
import PIL
import scipy

Define paths

In [33]:
base_dir = 'D:/ML projects/Kitty mood/data/images' 

Data augmentation procedure

In [34]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 280 images belonging to 6 classes.
Found 68 images belonging to 6 classes.


Load the pre-trained MobileNetV2 model and fine-tune it

In [35]:
base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(6, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\Kovi\AppData\Local\Temp\ipykernel_40628\4292523440.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')


Train the model

In [36]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)

Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.1414 - loss: 3.5558 - val_accuracy: 0.1324 - val_loss: 2.5158
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 629ms/step - accuracy: 0.2178 - loss: 2.7887 - val_accuracy: 0.2941 - val_loss: 2.0774
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 625ms/step - accuracy: 0.2449 - loss: 2.6673 - val_accuracy: 0.2353 - val_loss: 2.1044
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 626ms/step - accuracy: 0.2476 - loss: 2.8163 - val_accuracy: 0.2059 - val_loss: 2.0898
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 625ms/step - accuracy: 0.2673 - loss: 2.3072 - val_accuracy: 0.2941 - val_loss: 1.7480
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 629ms/step - accuracy: 0.3192 - loss: 2.0990 - val_accuracy: 0.2794 - val_loss: 1.8533
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 632ms/step - accuracy: 0.3761 - loss: 1.8227 - val_accuracy: 0.2500 - val_loss: 1.7821
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 633ms/step - accuracy: 0.3390 - loss: 2.0262 - val_accuracy: 0.2647 - val_

Save the model

In [47]:
models_dir = 'models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

model.save(os.path.join(models_dir, 'cat_mood_model.h5'))
print("Model saved successfully!")

Model saved successfully!


Load Model

In [48]:
model = tf.keras.models.load_model(os.path.join(models_dir, 'cat_mood_model.h5'))
print("Model loaded successfully!")

ValueError: Layer "dense_2" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 5, 5, 1280), dtype=float32, sparse=False, name=keras_tensor_2450>, <KerasTensor shape=(None, 5, 5, 1280), dtype=float32, sparse=False, name=keras_tensor_2451>]

Preprocess image

In [ ]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (150, 150))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

Predict mood

In [ ]:
def predict_mood(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    class_names = ['angry', 'calm', 'focused', 'groom', 'playful', 'sleepy']
    return class_names[np.argmax(prediction)]

Test the prediction function

In [ ]:
test_image_path = 'testimage/testimg.jpg'  # Ensure this path is correct and the file exists
mood = predict_mood(test_image_path)
print(f'The cat is {mood}.')